## 0. Import csv and lib

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
from collections import  Counter
import cv2
import tensorflow as tf
import time

from google.colab import drive
drive.mount('/content/drive')

# Pls change to your google drive train.csv path
df = pd.read_csv('/content/drive/My Drive/COMP5329S1A2Dataset/train.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. [This part is reserved for future caption nlp]

## 2. Data pre-processing

### 2.1. Multi label binary coding

In [3]:
labels = df['Labels'].tolist()
label_combination=[]
label_list=[]

for label in labels:
    num_label=[int(i) for i in label.split()]
    label_list.append(sorted(num_label))
    sort_label=[str(i)for i in sorted(num_label)]
    label_combination.append(' '.join(sort_label))

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(label_list)
y = mlb.transform(label_list)

print(y[:5])
print()
df.Labels.head()

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]]



0         1
1      1 19
2         1
3    8 3 13
4     8 3 7
Name: Labels, dtype: object

### 2.2. Prepare Train & Validation Data

In [0]:
h=32
w=32
file='Train_image_%d_%d'%(h,w)

# Pl change below as the npy file path in your google drive
DATA_DIR='/content/drive/My Drive/'
########################################################################



X=np.load(DATA_DIR+file+'.npy',allow_pickle=True)

from sklearn.model_selection import train_test_split
train_ratio = 0.8
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=train_ratio,random_state=123)

In [5]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(24000, 32, 32, 3)
(24000, 18)
(6000, 32, 32, 3)
(6000, 18)


## 3. CNN

### 3.1. Model Building

In [6]:
import seaborn as sns
from keras.optimizers  import adam
from keras.callbacks import Callback

from keras.utils import np_utils


from keras.models import Sequential 
from keras.layers import Dense,Conv2D, MaxPooling2D, Dropout, Flatten

def createModel():
    model =Sequential()
    model.add(Conv2D(32, (3 ,3),padding = 'same',activation = 'relu',input_shape = (32,32,3)))
    model.add(Conv2D(32, (3, 3) ,activation = 'relu' ))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    
    model.add(Conv2D(64, (3, 3), padding = 'same',activation = 'relu'))
    model.add(Conv2D(64, (3, 3), activation = 'relu' ))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    
    model.add(Conv2D(64, (3, 3), padding = 'same',activation = 'relu'))
    model.add(Conv2D(64, (3, 3), activation = 'relu' ))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(y_train.shape[1],activation='softmax'))
    
    
    return model



model = createModel()
AdamOpt = adam(lr=0.001)  
model.compile(optimizer=AdamOpt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary() 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)         

### 3.2. Model Training

In [0]:
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs={} ):
        if(epoch % 50 == 0):
            print("just finished",epoch)
            print('--------------------------')                        #just a funtion to show to accuracy after 5 epochs
            print('losss = ',logs.get('val_loss'))
            print('accuracy :',logs.get('accuracy'))
            print('val_acc = ',logs.get('val_accuracy'))
            return 

                                          #fit the model with thw datasets
cc=CustomCallback()

In [9]:
batch_size = 2560
epochs = 10 
hist = model.fit(X_train,y_train,batch_size =batch_size ,epochs =epochs,verbose=1,validation_data=(X_val, y_val),callbacks=[cc])

Train on 24000 samples, validate on 6000 samples
Epoch 1/10
24000/24000 [==============================] - 3s 143us/step - loss: 18210539756148.0547 - accuracy: 0.5200 - val_loss: 25738824087415.4648 - val_accuracy: 0.7578
just finished 0
--------------------------
losss =  25738824087415.465
accuracy : 0.52
val_acc =  0.7578333616256714
Epoch 2/10
24000/24000 [==============================] - 3s 139us/step - loss: 49915752335714.9844 - accuracy: 0.5197 - val_loss: 67238921985064.9609 - val_accuracy: 0.7578
Epoch 3/10
24000/24000 [==============================] - 3s 139us/step - loss: 123016982780968.9688 - accuracy: 0.5183 - val_loss: 155897362389838.5000 - val_accuracy: 0.7578
Epoch 4/10
24000/24000 [==============================] - 3s 139us/step - loss: 278320593456769.7188 - accuracy: 0.5061 - val_loss: 351640865109005.6250 - val_accuracy: 0.7578
Epoch 5/10
24000/24000 [==============================] - 3s 139us/step - loss: 597828249139718.7500 - accuracy: 0.5165 - val_loss: 70